In [281]:
import pandas as pd
from routing.routing_optimizer import RouteOptimizer
from datetime import datetime 
import folium
import openrouteservice as ors
import math

### Functions

In [282]:
def preprocessing(df_customer_sku_recommendation_raw, df_customer_dim_with_affinity_score_raw, df_stockpoint_dim_raw) :
    df_customer_sku_recommendation_raw['Stock_Point_ID'] = df_customer_sku_recommendation_raw['Stock_Point_ID'].astype(int)
    df_customer_dim_with_affinity_score_raw['Stock_Point_ID'] = df_customer_dim_with_affinity_score_raw['Stock_Point_ID'].astype(int)
    df_stockpoint_dim_raw['Stock_Point_ID'] = df_stockpoint_dim_raw['Stock_Point_ID'].astype(int)

    col1 = ['Latitude', 'Longitude','TotalSKUs', 'AvgSKUScore', 'TotalEstimatedVolume', 'RFcount', 'HighValueSKUs', 
            'HighValueAvgScore', 'HighValueTotalScore', 'HighValueEstimatedVolume', 'ExpressSKUs', 
            'CoreSKUs', 'CustomerAffinityScore_Raw', 'CustomerAffinityScore_Standardized', 'CustomerAffinityRank']

    for col in col1: 
        df_customer_dim_with_affinity_score_raw[col] = pd.to_numeric(df_customer_dim_with_affinity_score_raw[col], errors='coerce').fillna(0)

    col2 = ['EstimatedQuantity', 'CustomerSKUscore', 'CustomerSKUscoreStandardize', 'CustomerSKUscoreRank']
    for col in col2: 
        df_customer_sku_recommendation_raw[col] = pd.to_numeric(df_customer_sku_recommendation_raw[col], errors='coerce')

    df_customer_sku_recommendation_raw['LastDeliveredDate'] = pd.to_datetime(df_customer_sku_recommendation_raw['LastDeliveredDate'])
    # Get today's date
    today = pd.Timestamp.today()

    df_customer_sku_recommendation_raw['Recency'] = df_customer_sku_recommendation_raw['LastDeliveredDate'].apply(lambda x: (datetime.now() - x).days)
    df_customer_sku_recommendation_raw['Recency'] = df_customer_sku_recommendation_raw['Recency'].fillna(max(df_customer_sku_recommendation_raw['Recency']))
    df_stockpoint_dim_raw.rename(columns={'lattitude':'Latitude', 'longitude':'Longitude'}, inplace=True) 
    col3 = ['Latitude', 'Longitude']
    for col in col3: 
        df_stockpoint_dim_raw[col] = pd.to_numeric(df_stockpoint_dim_raw[col], errors='coerce')   

    # Replace invalid latitude values with NaN
    df_customer_dim_with_affinity_score_raw.loc[
        (df_customer_dim_with_affinity_score_raw['Latitude'] < -90) |
        (df_customer_dim_with_affinity_score_raw['Latitude'] > 90),
        'Latitude'
    ] = 0.0

    df_customer_dim_with_affinity_score_raw.loc[
        (df_customer_dim_with_affinity_score_raw['Longitude'] < -180) |
        (df_customer_dim_with_affinity_score_raw['Longitude'] > 180),
        'Longitude'
    ] = 0.0   


    return df_customer_sku_recommendation_raw, df_customer_dim_with_affinity_score_raw, df_stockpoint_dim_raw

In [283]:
def data_filter(df_customer_sku_recommendation, df_customer_dim_with_affinity_score, df_stockpoint_dim, stockpoint_id):
    df_customer_sku_recommendation = df_customer_sku_recommendation.copy().query(f'Stock_Point_ID == {stockpoint_id}')
    # Filter Recommendation
    df_customer_sku_recommendation = df_customer_sku_recommendation[df_customer_sku_recommendation['ProductTag'] != 'Standard-Inactive']
    df_customer_sku_recommendation = df_customer_sku_recommendation[df_customer_sku_recommendation['Medium'] != 'Never Purchased']

    ## Filter Product Bought recently
    df_customer_sku_recommendation = df_customer_sku_recommendation.query('Recency > 7')
    
    ## Selecting Customers who haven't bought any product in the last 3months
    df_select_customers = df_customer_sku_recommendation.groupby('CustomerID').Recency.min().reset_index().query('Recency <= 60')
    df_customer_sku_recommendation = df_customer_sku_recommendation.merge(df_select_customers[['CustomerID']], how = 'inner')
    
    # Select top 10 SKU by SKURank per customer
    df_customer_sku_recommendation = (
        df_customer_sku_recommendation
        .sort_values(['CustomerID','CustomerSKUscoreRank'])
        .groupby('CustomerID', group_keys=False)
        .head(5)
        .reset_index(drop=True) 
    )

    # df_customer_sku_recommendation = df_customer_sku_recommendation[df_customer_sku_recommendation['CustomerSKUscoreRank'] <= 10]

    # # Clipping Max Estimated Quantity to 10 qty
    df_customer_sku_recommendation['EstimatedQuantity_bck'] = df_customer_sku_recommendation['EstimatedQuantity']
    df_customer_sku_recommendation['EstimatedQuantity'] = df_customer_sku_recommendation['EstimatedQuantity'].apply(lambda x: 5 if int((x*0.90)) > 5 else int((x*0.90)) )


    df_stockpoint_dim = df_stockpoint_dim.query('Stock_Point_ID == 1647113').reset_index(drop=True)
 

    df_customer_dim = df_customer_dim_with_affinity_score.merge(df_customer_sku_recommendation[['CustomerID']].drop_duplicates(), how='inner', on='CustomerID')
    df_customer_sku_recommendation = df_customer_sku_recommendation.merge(df_customer_dim[['CustomerID','StateName', 'Region',
       'Latitude', 'Longitude', 'LGA', 'LCDA']].drop_duplicates(), how='inner', on='CustomerID')

    return df_customer_sku_recommendation.reset_index(drop=True), df_stockpoint_dim, df_customer_dim

In [284]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

def evaluate_unsupervised_clustering(df):
    # Usage:
    X = df[['Latitude', 'Longitude']].values
    labels = df['cluster'].values
    scores = {
        "Silhouette Score": silhouette_score(X, labels).round(2),
        "Davies-Bouldin Index": davies_bouldin_score(X, labels).round(2),
        "Calinski-Harabasz Score": calinski_harabasz_score(X, labels).round(2)
    }
    return scores

In [285]:
import folium
from clustering.plot_cluster import create_enhanced_cluster_map

def vis_and_save(df, df_causeway_stockpoint,   filename=f'./maps/default_clusters.html'):
    depot_location = [df_causeway_stockpoint.Latitude[0], df_causeway_stockpoint.Longitude[0]]
    depot_name = df_causeway_stockpoint.Stock_point_Name[0]
    map_clusters = create_enhanced_cluster_map(
        df,
        popup_cols=['CustomerID', 'LGA', 'LCDA'],
        tooltip_cols=['LGA', 'LCDA'], 
        zoom_start=10, 
        radius=8
    ).add_child(folium.Marker(location=depot_location, 
                            size = 10, 
                            tooltip=depot_name, 
                            icon=folium.Icon(color="green", 
                            icon="home")))  

    map_clusters.save(filename)
    return map_clusters

### Load Data

In [286]:
## Load Data
df_customer_sku_recommendation_raw = pd.read_feather('./input/customer_sku_recommendation.feather').rename(columns={'FCID':'Stock_Point_ID','CustomerId':'CustomerID'})
df_customer_dim_with_affinity_score_raw = pd.read_feather('./input/customer_dim_with_affinity_score.feather').rename(columns={'FCID':'Stock_Point_ID'})
df_stockpoint_dim_raw = pd.read_feather('./input/stockpoint_dim.feather')

In [287]:
df_customer_sku_recommendation, df_customer_dim_with_affinity_score, df_stockpoint_dim = preprocessing(df_customer_sku_recommendation_raw, 
                                                                                                       df_customer_dim_with_affinity_score_raw, 
                                                                                                       df_stockpoint_dim_raw)

In [288]:
causeway, causeway_stockpoint, causeway_customer_dim = data_filter(df_customer_sku_recommendation, 
                                                                   df_customer_dim_with_affinity_score, 
                                                                   df_stockpoint_dim, 
                                                                   stockpoint_id = 1647113)

print('Total Quantity b4 filter: ',df_customer_sku_recommendation.query('Stock_Point_ID == 1647113').EstimatedQuantity.sum()) #51,409 
print('Total Quantity: ',causeway.EstimatedQuantity.sum()) #15,613
print('Total Number of Customers: ', causeway_customer_dim.CustomerID.nunique()) #15,613


depot_location = [causeway_stockpoint.Latitude[0], causeway_stockpoint.Longitude[0]]

Total Quantity b4 filter:  51409
Total Quantity:  13277
Total Number of Customers:  875


### Routing

In [289]:
def main(df_sku_rec, df_customer_dim, df_stockpoint):
    """
    Main execution function demonstrating complete route optimization workflow
    """ 

    print("=" * 80)
    print("ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS")
    print("=" * 80)

    # STEP 1: Load or Generate Data
    print("\n1. Loading Data...")
    print("-" * 40)

    # Load data into optimizer 

    # df_sku_rec, df_customer_dim, df_stockpoint = causeway, df_customer_dim_with_affinity_score, causeway_stockpoint

    print(f"✓ Loaded {len(df_sku_rec)} SKU recommendations")
    print(f"✓ Loaded {len(df_customer_dim)} customer records")
    print(f"✓ Loaded {len(df_stockpoint)} stock points")


    # STEP 2: Initialize Route Optimizer
    print("\n2. Initializing Route Optimizer...")
    print("-" * 40)

    optimizer = RouteOptimizer(
        max_customers_per_route=20,
        max_volume_per_route=300,
        max_distance_km = 40
    )

    optimizer.load_data(df_sku_rec, df_customer_dim, df_stockpoint)
    print("✓ Route optimizer initialized")


    # STEP 3: Generate Routes for Stock Point 1647113
    print("\n3. Generating Optimized Routes...")
    print("-" * 40)
    stock_point_id = 1647113

    stock_point = optimizer.df_stockpoint[
            optimizer.df_stockpoint['Stock_Point_ID'] == stock_point_id
        ].iloc[0]
    stock_point_coords = (stock_point['Latitude'], stock_point['Longitude'])
        
    clustering_customers_df = optimizer.filter_customers_for_stockpoint(stock_point_id)

    df_clustering, n_clusters = optimizer.create_geographic_clusters(clustering_customers_df, clustering_method = 'divisive')

    routes = optimizer.generate_multi_trip_routes(stock_point_id, max_trips=5, clustering_method='divisive')
    df_routes = pd.DataFrame(routes)


    # STEP 4: Analyze Results
    print("\n4. Route Analysis & Results...")
    print("-" * 40)

    if routes:
        df_routes = pd.DataFrame(routes)
        
        print(f"✓ Generated {len(df_routes)} customer visits")
        print(f"✓ Number of trips: {df_routes['TripNumber'].max()}")
        print(f"✓ Total volume: {df_routes['EstimatedQuantity'].sum()} units")
        print(f"✓ Average priority score: {df_routes['PriorityScore'].mean():.3f}")
        
        # Trip-wise breakdown
        print("\nTrip Breakdown:")
        trip_summary = df_routes.groupby(['PLANID', 'TripNumber']).agg({
            'CustomerID': 'count',
            'EstimatedQuantity': 'sum',
            'CustomerAffinityRank': 'mean',
            'PriorityScore': 'mean'
        }).round(3)
        
        trip_summary.columns = ['Customers', 'Volume', 'Avg_Affinity_Rank', 'Avg_Priority_Score']
        print(trip_summary)
        
    # STEP 5: Display Route Details
    print("\n5. Detailed Route Plans...")
    print("-" * 40)

    for trip_num in sorted(df_routes['TripNumber'].unique()):
        trip_data = df_routes[df_routes['TripNumber'] == trip_num]
        print(f"\n*** TRIP {trip_num} - {trip_data.iloc[0]['PLANID']} ***")
        print(f"Customers: {len(trip_data)} | Volume: {trip_data['EstimatedQuantity'].sum()} | "
                f"Avg Priority: {trip_data['PriorityScore'].mean():.3f}")
        
        print("\nRoute Sequence:")
        for idx, row in trip_data.iterrows():
            print(f"  {row['Sequence']:2d}. {row['CustomerID']} | "
                    f"{row['CustomerName'][:20]:20s} | "
                    f"Vol: {row['EstimatedQuantity']:3d} | "
                    f"Priority: {row['PriorityScore']:.3f} | "
                    f"Rank: {row['CustomerAffinityRank']:3d}")
        
        # STEP 6: Export Results
        print("\n6. Exporting Results...")
        print("-" * 40)
        
        # Save detailed route plan
        output_filename = f'route_plan_SP{stock_point_id}.csv'
        df_routes.to_csv(output_filename, index=False)
        print(f"✓ Detailed route plan saved to: {output_filename}")
        
        # Save summary
        summary_filename = f'route_summary_SP{stock_point_id}.csv'
        trip_summary.to_csv(summary_filename)
        print(f"✓ Route summary saved to: {summary_filename}")
        
        # STEP 7: Key Insights
        print("\n7. Key Insights & Recommendations...")
        print("-" * 40)
        
        total_customers = len(df_routes)
        total_volume = df_routes['EstimatedQuantity'].sum()
        avg_customers_per_trip = total_customers / df_routes['TripNumber'].max()
        
        print(f"• Route Efficiency: {avg_customers_per_trip:.1f} customers per trip")
        print(f"• Volume Utilization: {(total_volume/df_routes['TripNumber'].max()/200)*100:.1f}% of capacity")
        print(f"• Priority Focus: {(df_routes['PriorityScore'] > 0.5).sum()} high-priority customers selected")
        
        # Geographic spread
        lat_range = df_routes['Latitude'].max() - df_routes['Latitude'].min()
        lon_range = df_routes['Longitude'].max() - df_routes['Longitude'].min()
        print(f"• Geographic Coverage: {lat_range:.3f}° lat × {lon_range:.3f}° lon")
        
        return df_clustering, df_routes, trip_summary
    
    else:
        print("❌ No routes generated. Check data and constraints.")
        return None, None


In [290]:
df_clustering, df_routes, trip_summary = main(df_sku_rec = causeway, df_customer_dim = causeway_customer_dim, df_stockpoint = causeway_stockpoint)

ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS

1. Loading Data...
----------------------------------------
✓ Loaded 4328 SKU recommendations
✓ Loaded 875 customer records
✓ Loaded 1 stock points

2. Initializing Route Optimizer...
----------------------------------------
✓ Route optimizer initialized

3. Generating Optimized Routes...
----------------------------------------

4. Route Analysis & Results...
----------------------------------------
✓ Generated 46 customer visits
✓ Number of trips: 5
✓ Total volume: 694 units
✓ Average priority score: 0.429

Trip Breakdown:
                          Customers  Volume  Avg_Affinity_Rank  \
PLANID        TripNumber                                         
SP1647113_T01 1                  18     300             17.278   
SP1647113_T02 2                   2      31             96.000   
SP1647113_T03 3                   6      82            216.833   
SP1647113_T04 4                   6      95            176.667   
SP1647113_T05 5       

In [291]:
causeway_push_recommendation = causeway.merge(df_clustering[['Stock_Point_ID','CustomerID', 'cluster']], how='inner', on =['Stock_Point_ID','CustomerID'] )

#### Evaluate Cluster

In [292]:
evaluate_unsupervised_clustering(df_clustering)

{'Silhouette Score': np.float64(0.37),
 'Davies-Bouldin Index': np.float64(0.77),
 'Calinski-Harabasz Score': np.float64(15658.62)}

#### Summary

In [293]:
causeway_push_recommendation_summary = causeway_push_recommendation.groupby('cluster').agg(
    LGAs = ('LGA', lambda x: x.unique().tolist()),
    LCDA = ('LCDA', lambda x: x.unique().tolist()),
    ncustomer = ('CustomerID','nunique'),
    totalQty = ('EstimatedQuantity','sum'),
    avgSKUscore = ('CustomerSKUscore','mean'),
).reset_index().sort_values(['ncustomer', 'avgSKUscore', 'totalQty'], ascending=[False, False, False])

causeway_push_recommendation_summary.head(10)

,cluster,LGAs,LCDA,ncustomer,totalQty,avgSKUscore
30,31,"[Apapa, Surulere, Ajeromi Ifelodun]","[Apapa - Sari-iganmu, Apapa - Ijora, Apapa - A...",20,300,5.789900
76,77,[Mushin],[Mushin - Mushin Market],20,365,5.760500
15,16,"[Surulere, Mushin]","[Surulere - Lawanson, Mushin - Ilasamaja Road,...",20,320,5.485253
78,79,[Ajeromi Ifelodun],[Ajeromi Ifelodun - Suru-alaba],20,338,5.085900
50,51,"[Lagos Island, Ajeromi Ifelodun]","[Lagos Island - Adeniji, Lagos Island - Tbs, L...",20,279,4.234330
40,41,"[Apapa, Ajeromi Ifelodun, Surulere, Illorin]","[Apapa - Amukoko, Apapa - Olodi, Ajeromi Ifelo...",19,292,6.140879
32,33,"[Surulere, Ajeromi Ifelodun]","[Surulere - Aguda, Surulere - Orile Iganmu, Aj...",19,278,5.637158
43,44,"[Ajeromi Ifelodun, Apapa]","[Ajeromi Ifelodun - Suru-alaba, Ajeromi Ifelod...",19,275,5.633370
25,26,"[Mushin, Surulere, Push - Surulere]","[Mushin - Mushin Market, Surulere - Ijesha, Su...",19,272,5.610211
35,36,[Surulere],"[Surulere - Aguda, Surulere - Ijesha]",19,277,5.581474


In [294]:
print(causeway_push_recommendation.columns.to_list())
df_routes.head(3)

['Stock_Point_ID', 'CustomerID', 'SKUID', 'ProductName', 'Output', 'LastDeliveredDate', 'InventoryCheck', 'ProductTag', 'Medium', 'EstimatedQuantity', 'CustomerSKUscore', 'CustomerSKUscoreStandardize', 'CustomerSKUscoreRank', 'Recency', 'EstimatedQuantity_bck', 'StateName', 'Region', 'Latitude', 'Longitude', 'LGA', 'LCDA', 'cluster']


,PLANID,TripNumber,Sequence,CustomerID,CustomerName,Latitude,Longitude,EstimatedQuantity,CumulativeVolume,CustomerAffinityRank,CustomerSKUscoreRank,ProductTags,Region,LGA,LCDA,cluster,PriorityScore
0,SP1647113_T01,1,1,3916211,Emmanuel Bright,6.453369,3.335517,25,25,1,1,Express,L2,Ajeromi Ifelodun,Ajeromi Ifelodun - Ajegunle,69,0.8900
1,SP1647113_T01,1,2,3899753,Anayochris Obasi,6.500004,3.386562,21,46,2,1,Express,L2,Lagos Mainland,Lagos Mainland - Yaba - Makoko,17,0.7565
2,SP1647113_T01,1,3,4582236,Bukola Tawaliu,6.521591,3.348540,22,68,3,2,"Express,Core,Standard",L2,Mushin,Mushin - Idi - Araba,52,0.7101


#### Map

In [295]:
map_name_all_rec = f'./recommendation_output/causeway_push_recommendation_cluster_{datetime.today().date()}.html'
map_clusters = vis_and_save(df_clustering, causeway_stockpoint, filename=map_name_all_rec)

map_clusters

In [296]:
# map_clusters = create_enhanced_cluster_map(
#         df_clustering,
#         popup_cols=['CustomerID', 'LGA', 'LCDA'],
#         tooltip_cols=['LGA', 'LCDA'], 
#         zoom_start=10 
#     ).add_child(folium.Marker(location=depot_location, 
#                             size = 10, 
#                             tooltip=causeway_stockpoint.Stock_point_Name[0], 
#                             icon=folium.Icon(color="green", 
#                             icon="home")))  

# map_clusters

#### Routing

In [297]:
# ======= BASE ROUTE MAP
# Visualize on Folium map
# m = folium.Map(location=depot_location, tiles="cartodbpositron", zoom_start=14)

# # Add customer markers
# for coord in coords:
#     folium.Marker(location=[coord[1], coord[0]]).add_to(m)  # [lat, lon] for folium

# # Add depot marker
# folium.Marker(location=depot_location, icon=folium.Icon(color="red")).add_to(m)

# # Display map
# m

In [308]:
causeway_push_recommendation_summary.head(20)


,cluster,LGAs,LCDA,ncustomer,totalQty,avgSKUscore
30,31,"[Apapa, Surulere, Ajeromi Ifelodun]","[Apapa - Sari-iganmu, Apapa - Ijora, Apapa - A...",20,300,5.789900
76,77,[Mushin],[Mushin - Mushin Market],20,365,5.760500
15,16,"[Surulere, Mushin]","[Surulere - Lawanson, Mushin - Ilasamaja Road,...",20,320,5.485253
78,79,[Ajeromi Ifelodun],[Ajeromi Ifelodun - Suru-alaba],20,338,5.085900
50,51,"[Lagos Island, Ajeromi Ifelodun]","[Lagos Island - Adeniji, Lagos Island - Tbs, L...",20,279,4.234330
40,41,"[Apapa, Ajeromi Ifelodun, Surulere, Illorin]","[Apapa - Amukoko, Apapa - Olodi, Ajeromi Ifelo...",19,292,6.140879
32,33,"[Surulere, Ajeromi Ifelodun]","[Surulere - Aguda, Surulere - Orile Iganmu, Aj...",19,278,5.637158
43,44,"[Ajeromi Ifelodun, Apapa]","[Ajeromi Ifelodun - Suru-alaba, Ajeromi Ifelod...",19,275,5.633370
25,26,"[Mushin, Surulere, Push - Surulere]","[Mushin - Mushin Market, Surulere - Ijesha, Su...",19,272,5.610211
35,36,[Surulere],"[Surulere - Aguda, Surulere - Ijesha]",19,277,5.581474


In [309]:
 # Select cluster 37
df_sel_clust = df_clustering.query('cluster in (44, 45, 32)')

# Ensure coordinates are in [longitude, latitude] for ORS
coords = [[lon, lat] for lat, lon in zip(df_sel_clust.Latitude, df_sel_clust.Longitude)]

# Convert depot_location to ORS format
# Assuming depot_location is [lat, lon], flip to [lon, lat]
vehicle_start = [depot_location[1], depot_location[0]]

In [312]:
depot_location = [causeway_stockpoint.Latitude[0], causeway_stockpoint.Longitude[0]]
depot_name = causeway_stockpoint.Stock_point_Name[0]

map_clusters_route = create_enhanced_cluster_map(
    df_sel_clust,
    popup_cols=['CustomerID', 'LGA', 'LCDA'],
    tooltip_cols=['LGA', 'LCDA'], 
    zoom_start=10, 
    radius=10
).add_child(folium.Marker(location=depot_location, 
                        size = 10, 
                        tooltip=depot_name, 
                        icon=folium.Icon(color="green", 
                        icon="home"))) 

# Print number of jobs
print("Number of customer locations:", len(coords))

# Define vehicles (capacity=5, starting and ending at depot)
vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[20]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[20]),
    ors.optimization.Vehicle(id=2, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[20]),
]

# Define jobs (each customer gets amount=[1])
jobs = [ors.optimization.Job(id=index, location=coord, amount=[1]) for index, coord in enumerate(coords)]

# Call ORS optimization API
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)

# Result is in 'optimized' JSON


Number of customer locations: 53


In [313]:
line_colors = ['green', 'orange', 'blue', 'yellow']
for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']], color=line_colors[route['vehicle']]).add_to(map_clusters_route)

map_clusters_route

#### Save Data

In [303]:
sel_columns = ['Stock_Point_ID', 
 'StateName', 'Region', 'Latitude', 'Longitude', 'LGA', 'LCDA', 'cluster', 
 'CustomerID', 'SKUID', 'ProductName', 'Output',
'LastDeliveredDate', 'Recency', 'InventoryCheck', 'ProductTag', 'Medium',
'EstimatedQuantity', 'CustomerSKUscoreRank']

causeway_push_recommendation_trip = causeway_push_recommendation[causeway_push_recommendation['cluster'].isin(list(df_sel_clust.cluster.unique()))][sel_columns]
causeway_push_recommendation_trip.rename(columns={'cluster':'TripID'}, inplace=True)

In [304]:
# causeway_push_recommendation_trip.head(10)

In [305]:
filename = f'./recommendation_output/causeway_push_recommendation_{datetime.today().date()}.xlsx'
map_name_trip = f'./recommendation_output/causeway_push_recommendation_trip_{datetime.today().date()}.html'
with pd.ExcelWriter(filename) as writer:
        causeway_push_recommendation_trip.to_excel(writer, sheet_name='Selected Trip', index=False)
        causeway_push_recommendation.to_excel(writer, sheet_name='All Recommendation', index=False)
        causeway_push_recommendation_summary.to_excel(writer, sheet_name='Recommendation Summary', index=False)

In [306]:
map_clusters_route.save(map_name_trip)